In [32]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

In [33]:
with open("df.pkl","rb") as fr:
    df = pickle.load(fr)
df = df.drop('이상치수치',axis=1)
K = df['행사가']
df = df.drop('행사가',axis=1)
df

,거래량,미결제약정,상대가격,잔존만기,rate,kvix,분류,이론하한,종가
0,78,2010,1.173773,0.023810,1.964,19.74,1,38.332852,0.172045
1,2,106,1.075958,0.023810,1.964,19.74,1,18.342202,0.083333
2,1,10,1.064866,0.023810,1.964,19.74,1,15.843371,0.064330
3,1,32,1.054000,0.023810,1.964,19.74,1,13.344540,0.054694
4,1,40,1.043354,0.023810,1.964,19.74,1,10.845709,0.044444
...,...,...,...,...,...,...,...,...,...
225044,1,1313,1.037342,1.936508,1.148,16.50,0,0.000000,0.086184
225045,1,10,1.459963,2.908730,1.148,16.50,0,0.000000,0.034630
225046,2,51,1.407821,2.908730,1.148,16.50,0,0.000000,0.037500
225047,5,311,1.313967,2.908730,1.148,16.50,0,0.000000,0.046667


In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
import time

In [35]:
df_ = df.drop(['이론하한','종가'],axis=1).copy()
scaler = MinMaxScaler()
scaler.fit(df_)
df_ = pd.DataFrame(scaler.transform(df_),columns=df.columns[:7])
df_

,거래량,미결제약정,상대가격,잔존만기,rate,kvix,분류
0,6.511452e-05,0.007142,0.255172,0.008011,1.000000,0.130817,1.0
1,8.456431e-07,0.000377,0.218227,0.008011,1.000000,0.130817,1.0
2,0.000000e+00,0.000036,0.214037,0.008011,1.000000,0.130817,1.0
3,0.000000e+00,0.000114,0.209933,0.008011,1.000000,0.130817,1.0
4,0.000000e+00,0.000142,0.205911,0.008011,1.000000,0.130817,1.0
...,...,...,...,...,...,...,...
225044,0.000000e+00,0.004665,0.203641,0.651535,0.507544,0.073924,0.0
225045,0.000000e+00,0.000036,0.363270,0.978638,0.507544,0.073924,0.0
225046,8.456431e-07,0.000181,0.343576,0.978638,0.507544,0.073924,0.0
225047,3.382572e-06,0.001105,0.308126,0.978638,0.507544,0.073924,0.0


In [36]:
%%time
ml_model = IsolationForest()#(n_clusters=2)
ml_model.fit(df_)

CPU times: total: 93.8 ms
Wall time: 234 ms


IsolationForest()

In [37]:
df_['anomaly_score'] = -ml_model.score_samples(df_)
df_['anomaly'] = ml_model.predict(df_.drop('anomaly_score',axis=1))
df_.loc[df_['anomaly']==1,'anomaly'] = 0
df_.loc[df_['anomaly']==-1,'anomaly'] = 1

In [38]:
df_.sample(frac=1).head(50)

,거래량,미결제약정,상대가격,잔존만기,rate,kvix,분류,anomaly_score,anomaly
112615,0.000000e+00,0.021121,0.340840,0.004005,0.135788,0.205970,0.0,0.388157,0
218041,2.536929e-06,0.000021,0.368412,0.679573,0.425468,0.145391,0.0,0.508681,1
139867,2.536929e-06,0.002537,0.151796,0.687583,0.105613,0.131343,1.0,0.539444,1
117106,1.522158e-05,0.015027,0.137890,0.124166,0.117079,0.187357,1.0,0.409631,0
72082,5.919502e-05,0.002640,0.214278,0.064085,0.662643,0.054609,0.0,0.380302,0
33939,0.000000e+00,0.007885,0.101414,0.491322,0.917924,0.024232,1.0,0.498986,0
7105,1.944979e-05,0.015574,0.084445,0.611482,0.904647,0.044601,1.0,0.528052,1
176070,9.406934e-03,0.024162,0.239544,0.013351,0.048884,0.116418,0.0,0.401629,0
104927,3.382572e-06,0.001343,0.103693,0.837116,0.043452,0.236172,1.0,0.602245,1
95519,0.000000e+00,0.079767,0.271500,0.220294,0.309596,0.481124,0.0,0.543101,1


In [39]:
df_['이론하한']=df['이론하한']
df_['종가']=df['종가']

In [40]:
df_['anomaly'].mean()

0.13087816431088342

In [41]:
df_.query('이론하한>종가').sample(frac=1)['anomaly'].mean()

0.08694263603979856

In [42]:
df_.to_csv('qr.csv')